## 0. Load imports 

In [3]:
## imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 0. Load data

In [4]:
## load data on 2020 crimes in DC
df = dc_crim_2020 = pd.read_csv("https://opendata.arcgis.com/datasets/f516e0dd7b614b088ad781b0c4002331_2.csv")
# dc_crim_2020.head()
# dc_crim_2020.shape
# dc_crim_2020.info()


In [168]:
df.OFFENSE.value_counts()

OFFENSE
THEFT/OTHER                   10927
THEFT F/AUTO                   8283
MOTOR VEHICLE THEFT            3260
ROBBERY                        1998
ASSAULT W/DANGEROUS WEAPON     1632
BURGLARY                       1444
HOMICIDE                        198
SEX ABUSE                       178
ARSON                            13
Name: count, dtype: int64

In [ ]:
df.OFFENSE.value_counts()

In [14]:
## Use tab to autocomplete
## Use shift-tab to see variables in a function.

df.OFFENSE.value_counts()
df.WARD.value_counts()

In [99]:
df.X + 1

0       -76.004500
1       -75.972651
2       -75.972226
3       -76.023981
4       -76.044182
           ...    
27928   -76.024529
27929   -75.982318
27930   -76.053302
27931   -76.028143
27932   -75.961475
Name: X, Length: 27933, dtype: float64

#### Warm-up: Ways to add one to the column

In [197]:
df.X + 1

0       -76.004500
1       -75.972651
2       -75.972226
3       -76.023981
4       -76.044182
           ...    
27928   -76.024529
27929   -75.982318
27930   -76.053302
27931   -76.028143
27932   -75.961475
Name: X, Length: 27933, dtype: float64

In [ ]:
plus_1 = lambda x: x + 1

def plus_1_v2(x):
    return x + 1

In [198]:
df.X.apply(plus_1)

0       -76.004500
1       -75.972651
2       -75.972226
3       -76.023981
4       -76.044182
           ...    
27928   -76.024529
27929   -75.982318
27930   -76.053302
27931   -76.028143
27932   -75.961475
Name: X, Length: 27933, dtype: float64

In [106]:
## To see what a sub-dataframe looks like, iterate with a for-loop

for i, ddf in df.groupby("WARD"):
    break
ddf[["WARD", "OFFENSE", "SHIFT"]]

,WARD,OFFENSE,SHIFT
12,1,THEFT/OTHER,DAY
23,1,THEFT F/AUTO,MIDNIGHT
27,1,THEFT/OTHER,EVENING
28,1,THEFT F/AUTO,DAY
30,1,THEFT/OTHER,EVENING
...,...,...,...
27899,1,THEFT F/AUTO,DAY
27915,1,MOTOR VEHICLE THEFT,MIDNIGHT
27916,1,THEFT F/AUTO,EVENING
27920,1,THEFT F/AUTO,DAY


## 1. Aggregation

### Example of grouping by one variable and doing one aggregation 

**Task**: find and print the number of unique offense types (`OFFENSE`) by ward (`WARD`)

In [9]:
## Explore data by simulating a loop / groupby
for i, ddf in df.groupby("WARD"):
    break
    
y = ddf[["WARD", "OFFENSE"]].OFFENSE.unique()
len(y)

9

In [10]:
## Translate exploration code into a function
def get_unique_num(X):
    y = X.unique()
    return len(y)

In [11]:
df.groupby(["WARD"]).agg( 
    {"OFFENSE": get_unique_num}
).reset_index() #the iloc shows the first five

,WARD,OFFENSE
0,1,9
1,2,9
2,3,8
3,4,8
4,5,9
5,6,9
6,7,9
7,8,9


In [12]:
%%time
uniq_off_byward = U = df.groupby("WARD").agg(
    {"OFFENSE":"nunique"}
    ).reset_index()
U

CPU times: user 11.8 ms, sys: 3.13 ms, total: 15 ms
Wall time: 15.3 ms


,WARD,OFFENSE
0,1,9
1,2,9
2,3,8
3,4,8
4,5,9
5,6,9
6,7,9
7,8,9


### Example of grouping by one variable and providing two summaries of the same variable

**Task**: previous showed number of offenses by ward but want to find out content of offenses in each ward; create an aggregation that summarizes both the number of unique offenses by ward and what those offenses are -- for instance, by pasting the unique offenses in that ward separated by the ";" (e.g., Theft; Burglary;...)

*Hint*: you can use the join command to paste together a list separated by some delimiter. 
    - The syntax, if we are using the comma delimiter, is: ", ".join(nameoflist)

In [13]:
def get_uniq_offs(X):
    uniq_offs = sorted( X.unique() )
    uniq_offs = "; ".join( uniq_offs )
    return uniq_offs

In [14]:
A = uniq_off_valcount_byward = df.groupby("WARD").agg(
        {"OFFENSE": ["nunique", get_uniq_offs]}
        )
A

OFFENSE                                                   
     nunique                                      get_uniq_offs
WARD                                                           
1          9  ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
2          9  ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
3          8  ASSAULT W/DANGEROUS WEAPON; BURGLARY; HOMICIDE...
4          8  ASSAULT W/DANGEROUS WEAPON; BURGLARY; HOMICIDE...
5          9  ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
6          9  ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
7          9  ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
8          9  ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...

### Example of grouping by two variables 

**Task**: group by ward (`WARD`) and police shift (`SHIFT`) and find the offense that is most common in that ward and shift

In [15]:
# Explore the data
for i, ddf in df.groupby(["WARD", "SHIFT"]):
    break
    
x = ddf.OFFENSE
x = x.value_counts()
x.index[0]

'THEFT/OTHER'

In [16]:
df.groupby(["WARD", "SHIFT"])["OFFENSE"].agg(pd.Series.mode).reset_index()

,WARD,SHIFT,OFFENSE
0,1,DAY,THEFT/OTHER
1,1,EVENING,THEFT/OTHER
2,1,MIDNIGHT,THEFT/OTHER
3,2,DAY,THEFT/OTHER
4,2,EVENING,THEFT/OTHER
5,2,MIDNIGHT,THEFT/OTHER
6,3,DAY,THEFT/OTHER
7,3,EVENING,THEFT/OTHER
8,3,MIDNIGHT,THEFT/OTHER
9,4,DAY,THEFT F/AUTO


In [17]:
B = ward_shift_top_offense = df.groupby(["WARD", "SHIFT"]).agg(
        {
            "OFFENSE": lambda x: x.value_counts().index[0]
        }
        ).reset_index()
B

,WARD,SHIFT,OFFENSE
0,1,DAY,THEFT/OTHER
1,1,EVENING,THEFT/OTHER
2,1,MIDNIGHT,THEFT/OTHER
3,2,DAY,THEFT/OTHER
4,2,EVENING,THEFT/OTHER
5,2,MIDNIGHT,THEFT/OTHER
6,3,DAY,THEFT/OTHER
7,3,EVENING,THEFT/OTHER
8,3,MIDNIGHT,THEFT/OTHER
9,4,DAY,THEFT F/AUTO


## Summarizing over all rows or all columns (without grouping)

We can also use the `apply` function to summarize rows or columns efficiently

**Task**: find the mean lat and longitude in one line of code

In [63]:
df[["LATITUDE", "LONGITUDE"]].apply("mean", axis = 0)

LATITUDE     38.907613
LONGITUDE   -77.007049
dtype: float64

## 2. Creating new columns/transforming their type 

### Simple filtering

`np.where` is in the numpy package (aliased as `np`) and operates similar to `ifelse` in R

There is a built in pandas way to emulate this.

**Task**: create an indicator `is_theft` for any offense that contains the word "THEFT"
    
**Task**: create an indicator `is_theft_notmotor` for any offense that contains the word "THEFT" but does not contain the word "MOTOR"

In [18]:
df["is_theft"] = df.OFFENSE.str.contains("THEFT")

In [69]:
df["is_theft_notmotor"] = ( df.OFFENSE.str.contains("THEFT") &
                        ~df.OFFENSE.str.contains("MOTOR")  )

In [159]:
df.is_theft & (~df.OFFENSE.str.contains("MOTOR"))

0        False
1        False
2        False
3         True
4        False
         ...  
27928    False
27929    False
27930     True
27931    False
27932     True
Length: 27933, dtype: bool

### np.select

**Task**: create a new variable, `offense_summary`, where you:
        
- Recode theft offenses that use a gun or knife as the method (`METHOD`) as: violent theft
- Recode non-theft offenses that use a gun or knife as the method as: violent other
- Recode all other as non-violent 

In [ ]:
conditions = [
    df.is_theft & df.METHOD.isin(["GUN", "KNIFE"]),
    ~df.is_theft & df.METHOD.isin(["GUN", "KNIFE"]),
        ]

code_to_cats = [
    "violent theft", "violent other"
        ]

df["offense_summary"] = np.select(conditions, code_to_cats, 
                                 default = "non-violent")

df.offense_summary.value_counts()

### Transforming column types

**Task**: check the type of the `START_DATE` column

In [5]:
df.START_DATE.dtype

dtype('O')

**Task**: recast the `START_DATE` column to datetime, calling this `START_DATE_CLEAN`; coerce errors rather than cleaning the string; check the type of that new col

In [6]:
df["START_DATE_CLEAN"] = pd.to_datetime(df.START_DATE)

In [7]:
df.START_DATE_CLEAN.min(), df.START_DATE_CLEAN.max()

(Timestamp('1940-02-17 00:00:16+0000', tz='UTC'),
 Timestamp('2021-01-01 02:30:21+0000', tz='UTC'))

In [ ]:
## Use this to generate groups of year-month
df.START_DATE_CLEAN.dt.to_period("M")

## 3. Row and column filtering 

### Row subsetting

**Task**: filter to crime reports about theft using the `is_theft` definition and that are in ward 3

In [22]:
theft_W3 = df[df.is_theft & (df.WARD == 3)]
theft_W3.head()

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,VOTING_PRECINCT,LATITUDE,LONGITUDE,BID,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID,START_DATE_CLEAN,is_theft
11,-77.051908,38.924199,20170566,2020/11/30 20:53:51+00,EVENING,OTHERS,THEFT/OTHER,2600 - 2649 BLOCK OF CONNECTICUT AVENUE NW,395499.0,139592.0,...,Precinct 136,38.924191,-77.051906,NaN,2020/11/30 16:45:06+00,2020/11/30 17:30:13+00,398217704,NaN,2020-11-30 16:45:06+00:00,True
18,-77.083768,38.960042,20121641,2020/08/26 03:29:34+00,MIDNIGHT,OTHERS,THEFT F/AUTO,5300 - 5399 BLOCK OF 43RD STREET NW,392740.0,143573.0,...,Precinct 32,38.960035,-77.083765,NaN,2020/08/25 22:30:03+00,2020/08/26 01:00:04+00,398217732,NaN,2020-08-25 22:30:03+00:00,True
28,-77.060408,38.938806,20160695,2020/11/10 05:15:06+00,MIDNIGHT,OTHERS,THEFT/OTHER,3700 - 3899 BLOCK OF CONNECTICUT AVENUE NW,394763.0,141214.0,...,Precinct 27,38.938799,-77.060406,NaN,2020/11/10 03:47:06+00,2020/11/10 05:15:44+00,398217791,NaN,2020-11-10 03:47:06+00:00,True
59,-77.074228,38.962589,20035501,2020/02/27 22:44:09+00,EVENING,OTHERS,THEFT/OTHER,5420 - 5499 BLOCK OF CONNECTICUT AVENUE NW,393567.0,143855.0,...,Precinct 50,38.962581,-77.074226,NaN,2020/02/27 18:50:10+00,2020/02/27 22:00:06+00,398217992,NaN,2020-02-27 18:50:10+00:00,True
74,-77.056309,38.933503,20001654,2020/01/03 20:09:51+00,EVENING,OTHERS,THEFT F/AUTO,2700 - 2899 BLOCK OF MACOMB STREET NW,395118.0,140625.0,...,Precinct 34,38.933495,-77.056307,NaN,2020/01/03 19:38:39+00,2020/01/03 20:10:05+00,398218125,NaN,2020-01-03 19:38:39+00:00,True


### Column subsetting
**Task**: select two columns--`START_DATE` and `END_DATE`--and print the head of the dataframe

In [19]:
cols = [c for c in df.columns if "DATE" in c]
df[cols].head()

,START_DATE,END_DATE,START_DATE_CLEAN
0,2020/10/27 20:26:31+00,2020/10/27 20:37:29+00,2020-10-27 20:26:31+00:00
1,2020/06/29 23:30:04+00,2020/06/30 11:00:09+00,2020-06-29 23:30:04+00:00
2,2020/04/27 22:49:30+00,NaN,2020-04-27 22:49:30+00:00
3,2020/07/12 17:36:13+00,2020/07/12 18:10:45+00,2020-07-12 17:36:13+00:00
4,2020/09/09 01:30:07+00,2020/09/10 23:00:57+00,2020-09-09 01:30:07+00:00


**Task**: filter reports of crime in `START_DATE_CLEAN` before May 2020 and that are located in ward 1; select the ward and `START_DATE` column and print a random sample of rows

In [23]:
df1 = df[(df.START_DATE_CLEAN < "2020/05/01") & (df.WARD == 1)]
df1[["WARD", "START_DATE"]].sample(10)

,WARD,START_DATE
4583,1,2020/03/04 23:58:07+00
17613,1,2020/02/21 22:29:03+00
16308,1,2020/03/21 06:38:59+00
13041,1,2020/03/02 19:28:41+00
5290,1,2020/02/20 05:11:46+00
13982,1,2020/02/23 09:45:49+00
18506,1,2020/04/12 19:36:24+00
20352,1,2020/03/13 16:00:53+00
26708,1,2020/02/06 21:20:53+00
12249,1,2020/04/12 16:24:09+00
